## Setup

The cell below downloads the code from Github and install necessary dependencies.

### Import Packages

In [1]:
%load_ext autoreload
%autoreload 2

import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "2"


In [2]:
import json
import os
import time
from typing import Any

import jax
import jax.numpy as jnp
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from absl import logging
from matplotlib.colors import ListedColormap
from deeprte.model.engine import RteEngine

from rte_dataset.builders import pipeline

logging.set_verbosity(logging.INFO)

jax.local_devices()

[CudaDevice(id=0), CudaDevice(id=1), CudaDevice(id=2), CudaDevice(id=3)]

### Utility functions

In [3]:
def rmse(pred, target):
    return np.sqrt(np.mean((pred - target) ** 2) / np.mean(target**2))


def get_normalization_ratio(psi_range, boundary_range):
    psi_range = float(psi_range.split(" ")[-1])
    boundary_range = float(boundary_range.split(" ")[-1])
    return psi_range / boundary_range


def jnp_to_np(output: dict[str, Any]) -> dict[str, Any]:
    """Recursively changes jax arrays to numpy arrays."""
    for k, v in output.items():
        if isinstance(v, dict):
            output[k] = jnp_to_np(v)
        elif isinstance(v, jnp.ndarray):
            output[k] = np.array(v)
    return output


def plot_phi(r, phi_pre, phi_label):
    fig, _axs = plt.subplots(nrows=1, ncols=3, figsize=(20, 6))
    fig.subplots_adjust(hspace=0.3)
    axs = _axs.flatten()

    viridis = matplotlib.colormaps["viridis"](np.linspace(0, 1.2, 128))
    cs_1 = axs[0].contourf(
        r[..., 0], r[..., 1], phi_label, cmap=ListedColormap(viridis)
    )
    axs[0].set_title(r"Exact $f(x,v)$", fontsize=20)
    axs[0].tick_params(axis="both", labelsize=15)
    cbar = fig.colorbar(cs_1)
    cbar.ax.tick_params(labelsize=16)

    cs_2 = axs[1].contourf(r[..., 0], r[..., 1], phi_pre, cmap=ListedColormap(viridis))
    axs[1].set_title(r"Predict $f(x,v)$", fontsize=20)
    axs[1].tick_params(axis="both", labelsize=15)
    cbar = fig.colorbar(cs_2)
    cbar.ax.tick_params(labelsize=16)

    cs_3 = axs[2].contourf(
        r[..., 0],
        r[..., 1],
        abs(phi_pre - phi_label),
        cmap=ListedColormap(viridis),
    )
    axs[2].set_title(r"Absolute error", fontsize=20)
    axs[2].tick_params(axis="both", labelsize=15)
    cbar = fig.colorbar(cs_3)
    cbar.ax.tick_params(labelsize=16)

    plt.tight_layout()

## Load Dataset

In [4]:
# Path to the dataset
DATA_DIR = "/workspaces/deeprte/data/raw_data/train/g0.8-sigma_a3-sigma_t6"
DATA_NAME = ["g0.8-sigma_a3-sigma_t6.mat"]

data_pipeline = pipeline.DataPipeline(DATA_DIR, DATA_NAME)
raw_feature_dict = data_pipeline.process()
num_examples = raw_feature_dict["shape"]["num_examples"]

del data_pipeline

jax.tree.map(lambda x: x.shape, raw_feature_dict["functions"])

{'boundary': (1000, 160, 12),
 'boundary_scattering_kernel': (1000, 160, 12, 24),
 'psi_label': (1000, 40, 40, 24),
 'scattering_kernel': (1000, 40, 40, 24, 24),
 'self_scattering_kernel': (1000, 24, 24),
 'sigma': (1000, 40, 40, 2)}

## Import Pre-trained model

#### Load model config

In [5]:
import imp

MODEL_DIR = "/workspaces/deeprte/ckpts/g0.8-infer/params"
CONFIG_PATH = "/workspaces/deeprte/deeprte/configs/default.py"

config_module = imp.load_source("config_module", CONFIG_PATH)
config = config_module.get_config()
config.load_parameters_path = MODEL_DIR

config

/tmp/ipykernel_6534/736015114.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


Config(seed=42, dataset_type='tfds', grain_worker_count=4, grain_worker_buffer_size=1, dataset_name='rte/g0.5-sigma_a3-sigma_t6', data_dir='/workspaces/deeprte/data/tfds', train_split='train[:80%]', eval_split='train[80%:]', enable_data_shuffling=True, data_shuffle_seed=42, per_device_batch_size=1, global_batch_size=8, collocation_size=128, repeat_batch=1, eval_batch_size=4, num_train_steps=500001, micro_steps=2, log_every_steps=1000, eval_every_steps=50000, optimizer='adam', learning_rate=0.001, schedule='cosine_decay', decay_steps=500001, save_checkpoints=True, checkpoint_every_steps=10000, async_checkpointing=True, enable_checkpoint_standard_logger=True, load_parameters_path='/workspaces/deeprte/ckpts/g0.8-infer/params', load_full_state_path='', enable_single_replica_ckpt_restoring=False, enable_emergency_checkpoint=False, local_checkpoint_dir='', local_checkpoint_every_steps=10000, position_coords_dim=2, velocity_coords_dim=2, coeffs_fn_dim=2, num_heads=2, qkv_dim=64, optical_depth

#### Load model parameters

In [6]:
rte_engine = RteEngine(config)

INFO:absl:Devices: [CudaDevice(id=0), CudaDevice(id=1), CudaDevice(id=2), CudaDevice(id=3)]
INFO:absl:Number of devices: 4
INFO:absl:Decided on mesh: [[[CudaDevice(id=0)]
  [CudaDevice(id=1)]
  [CudaDevice(id=2)]
  [CudaDevice(id=3)]]]
INFO:absl:Mesh shape: (1, 4, 1)
INFO:absl:Loading decode params from /workspaces/deeprte/ckpts/g0.8-infer/params
INFO:absl:restoring params from /workspaces/deeprte/ckpts/g0.8-infer/params
INFO:absl:Restoring checkpoint from /workspaces/deeprte/ckpts/g0.8-infer/params.
INFO:absl:Finished restoring checkpoint from /workspaces/deeprte/ckpts/g0.8-infer/params.
INFO:absl:Skipping global process sync, barrier name: Checkpointer:restore.params
INFO:absl:Number of model params=37954


#### Construct model runner

In [7]:

def get_normalization_ratio(psi_range, boundary_range):
    psi_range = float(psi_range.split(" ")[-1])
    boundary_range = float(boundary_range.split(" ")[-1])
    return psi_range / boundary_range

normalization_dict = {"psi_min": "2.955854e-25", "psi_range": "0.13107203", "boundary_min": "0.0", "boundary_range": "0.1616703"}
normalization_ratio = get_normalization_ratio(
    normalization_dict["psi_range"],
    normalization_dict["boundary_range"],
)

normalization_ratio

0.81073660406395

## Predict and Evaluate

#### Predict and Evaluate i-th example in the dataset

#### Run prediction

In [8]:
i = np.random.randint(100)

logging.info("Predicting example %d/%d", i + 1, num_examples)

benchmark = True

feature_dict = {
    "functions": jax.tree.map(lambda x: x[i : i + 1], raw_feature_dict["functions"]),
    "grid": raw_feature_dict["grid"],
    "shape": raw_feature_dict["shape"],
}

# Run the model.
logging.info("Running rte engine...")
processed_feature_dict = rte_engine.process_features(feature_dict)

jax.tree.map(lambda x: x.shape, processed_feature_dict)


t_0 = time.time()
prediction = rte_engine.predict(processed_feature_dict)
t_diff = time.time() - t_0

if benchmark:
    t_0 = time.time()
    rte_engine.predict(processed_feature_dict)
    t_diff = time.time() - t_0
    logging.info(
        "Total JAX model predict time " "(excludes compilation time): %.6fs",
        t_diff,
    )

psi_shape = feature_dict["functions"]["psi_label"].shape
t_0 = time.time()
predicted_psi = (
    prediction
    .reshape(1, -1)  # reshape multi_devices to single device
    .reshape(psi_shape)
)
if normalization_ratio:
    predicted_psi = predicted_psi * normalization_ratio

predicted_phi = jnp.sum(
    predicted_psi * feature_dict["grid"]["velocity_weights"],
    axis=-1,
)
t_diff = time.time() - t_0

# Compute metrics.
metrics = {}
psi_label = feature_dict["functions"]["psi_label"]
phi_label = np.sum(psi_label * feature_dict["grid"]["velocity_weights"], axis=-1)
psi_rmse = rmse(predicted_psi, psi_label)
phi_rmse = rmse(predicted_phi, phi_label)
logging.info("RMSE of psi: %f, RMSE of phi: %f\n", psi_rmse, phi_rmse)

INFO:absl:Predicting example 7/1000
INFO:absl:Running rte engine...
INFO:absl:Running predict with shape(feat) = {'boundary': (1, 1920), 'boundary_coords': (1, 1920, 4), 'boundary_weights': (1, 1920), 'phase_coords': (1, 38400, 4), 'position_coords': (1, 1600, 2), 'scattering_kernel': (1, 38400, 24), 'self_scattering_kernel': (1, 24, 24), 'sigma': (1, 1600, 2), 'velocity_coords': (1, 24, 2), 'velocity_weights': (1, 24)}
INFO:absl:Output shape was (1, 38400)
INFO:absl:Running predict with shape(feat) = {'boundary': (1, 1920), 'boundary_coords': (1, 1920, 4), 'boundary_weights': (1, 1920), 'phase_coords': (1, 38400, 4), 'position_coords': (1, 1600, 2), 'scattering_kernel': (1, 38400, 24), 'self_scattering_kernel': (1, 24, 24), 'sigma': (1, 1600, 2), 'velocity_coords': (1, 24, 2), 'velocity_weights': (1, 24)}
INFO:absl:Output shape was (1, 38400)
INFO:absl:Total JAX model predict time (excludes compilation time): 2.535386s
INFO:absl:RMSE of psi: 0.042864, RMSE of phi: 0.041453



### Visualize result

In [ ]:
plot_phi(
    feature_dict["grid"]["position_coords"].reshape(*psi_shape[1:-1], -1),
    predicted_phi[0],
    phi_label[0],
)